In [ ]:
import chess
import chess.svg

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import spark


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean, sum as _sum, count

# Initialiser une session Spark
spark = SparkSession.builder.appName("Lichess Analysis").getOrCreate()

# Charger les données depuis le fichier CSV (remplacez par le chemin du fichier si besoin)
data_path = "data/Sept_20_analysis.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Aperçu des données
df.show(5)
print("\nDimensions du dataset :", (df.count(), len(df.columns)))


In [ ]:
# Définition des catégories de niveau en fonction de l'ELO
def categorize_elo(column):
    return  when((col(column) >= 1200) & (col(column) < 1500),  "occasional player") \
           .when((col(column) >= 1500) & (col(column) < 1800),  "good club player") \
           .when((col(column) >= 1800) & (col(column) < 2000),  "very good club player") \
           .when((col(column) >= 2000) & (col(column) < 2400),  "national and international level (IM)") \
           .when((col(column) >= 2400) & (col(column) <= 2800), "GMI, World Champions") \
           .otherwise("Unknown")

# Ajouter les colonnes pour les catégories des joueurs
df = df.withColumn("White_elo_category", categorize_elo("WhiteElo"))
df = df.withColumn("Black_elo_category", categorize_elo("BlackElo"))
df = df.withColumn("Average_elo_category", categorize_elo(((col("WhiteElo") + col("BlackElo")) / 2)))

In [ ]:
# Filtrer les parties Blitz
blitz_games = df.filter(col("Game_type") == "Blitz")

# Calcul des taux par catégorie (ELO moyen)
rates_by_average_elo = blitz_games.groupBy("Average_elo_category").agg(
    (_sum(col("Black_blunders")) + _sum(col("White_blunders"))) / _sum(col("Total_moves")).alias("Blunder_rate"),
    (_sum(col("Black_mistakes")) + _sum(col("White_mistakes"))) / _sum(col("Total_moves")).alias("Mistake_rate"),
    (_sum(col("Black_inaccuracies")) + _sum(col("White_inaccuracies"))) / _sum(col("Total_moves")).alias("Inaccuracy_rate")
)

print("\nTaux par catégorie basée sur l'ELO moyen pour les parties Blitz :")
rates_by_average_elo.show()

# Filtrer les parties où les deux joueurs sont dans la même catégorie
same_category_blitz_games = blitz_games.filter(col("White_elo_category") == col("Black_elo_category"))
rates_by_same_category = same_category_blitz_games.groupBy("White_elo_category").agg(
    (_sum(col("Black_blunders")) + _sum(col("White_blunders"))) / _sum(col("Total_moves")).alias("Blunder_rate"),
    (_sum(col("Black_mistakes")) + _sum(col("White_mistakes"))) / _sum(col("Total_moves")).alias("Mistake_rate"),
    (_sum(col("Black_inaccuracies")) + _sum(col("White_inaccuracies"))) / _sum(col("Total_moves")).alias("Inaccuracy_rate")
)

print("\nTaux par catégorie lorsque les deux joueurs sont dans la même catégorie (parties Blitz) :")
rates_by_same_category.show()

# Sauvegarder les résultats
rates_by_average_elo_path = "rates_by_average_elo.csv"
rates_by_same_category_path = "rates_by_same_category.csv"
rates_by_average_elo.write.csv(rates_by_average_elo_path, header=True)
rates_by_same_category.write.csv(rates_by_same_category_path, header=True)

print(f"\nLes résultats ont été sauvegardés dans {rates_by_average_elo_path} et {rates_by_same_category_path}.")